# 15 - Lidando com dimensões

In [8]:
import thinkbayes

## 15.1 - Bactérias do umbigo

- Projeto com objetivo de identificar as especies de bactérias que pode ser encontrado em umbigo humanos;
- É de interesse do microbioma humano, o conjunto de migroorganismo que vive na pele humana e partes do corpo;

- Os pesquisadores coletaram amostras de 60 volumtários;
- Usou o método multiplex pyrosequencing para extrair e sequenciar fragmentos de rDNA 16S;

Podemos usar esses dados para responder a várias questões relacionadas:

- Com base no número de espécies observadas, podemos estimar o número total de espécies no ambiente?
- Podemos estimar a prevalência de cada espécie, ou seja, a fração da população total pertencente a cada espécie?
- Se estamos planejando coletar amostras adicionais, podemos prever quantas novas espécies poderemos descobrir?
- Quantas leituras adicionais são necessárias para aumentar a fração de espécies observadas até um determinado limite?

**Problema das Espécies Invisíveis**

## 15.2 - Leões, tigres e ursos

Problema simples onde conhecemos 3 espécies;
- 3 Leões, 2 Tigres e 1 Urso;

Chance igual de observar qualquer animal, o de vermos cada espécie é dada pela distribuição multinomial;
A verossimilhança de ver 3 leões, 2 tigres e um urso é proporcional a
```python
    p_lion**3 * p_tiger**2 * p_bear**1
```

- **Distribuição Beta**: Uma abordagem atentadora, mas não correta;

```python
    beta = thinkbayes.Beta()
    beta.Update((3, 3))
    print beta.MaximumLikelihood()
```
- `p_lion = 50%`
- `p_tiger = 33%`
- `p_bear = 17%`

**Problemas**
1. A a priori não é adequada para esse tipo de problema:
    - Distribuição uniforme para cada espécie;
2. As distribuições para cada espécie não pode ser independente:
    - Precisamos de uma distribuição conjunta;
    
**Solução**
- Usar uma distribuição Dirichlet, assim, resolvendo os dois problemas;
    - Descreve a distribuição conjunta de `p_lion`, `p_tiger` e `p_bear`.
    - É a generalização multidimensional da distribuição beta;
    - É descrita por n parâmetros, escritos de `α1` a `αn`.
    - A distribuição marginal para cada prevalência é uma distribuição beta;

In [6]:
import numpy

class Dirichlet(object):

    def __init__(self, n):
        self.n = n
        self.params = numpy.ones(n, dtype=numpy.int)
        
    def MarginalBeta(self, i):
        alpha0 = self.params.sum()
        alpha = self.params[i]
        return thinkbayes.Beta(alpha, alpha0-alpha)
    
    def Update(self, data):
        m = len(data)
        self.params[:m] += data

In [3]:
dirichlet = thinkbayes.Dirichlet(3)
for i in range(3):
    beta = dirichlet.MarginalBeta(i)
    print(beta.Mean())

0.3333333333333333
0.3333333333333333
0.3333333333333333


```python
data = [3, 2, 1]
dirichlet = Dirichlet(3)
dirichlet.Update(data)

for i in range(3):
    beta = dirichlet.MarginalBeta(i)
    pmf = beta.MakePmf()
    print(i, pmf.Mean())

0 0.44 
1 0.33
2 0.22    
```


![](post_15_2.png)

Distribuição das prevalências para três espécies

## 15.3 - A versão hierárquica

**Problema simples:** Se soubermos quantas espécies existem, podemos estimar a prevalência de cada uma.

**Problema original:** Estimando o número total de espécies
- Definiu **Meta-Suite**, que é uma Suite que contém outras Suites como hipóteses.
- Nível superior contém hipóteses sobre o número de espécies; 
- Nível inferior contém hipóteses sobre prevalências.

In [19]:
class Species(thinkbayes.Suite):

    def __init__(self, ns):
        hypos = [thinkbayes.Dirichlet(n) for n in ns]
        thinkbayes.Suite.__init__(self, hypos)
        
    def Update(self, data):
        thinkbayes.Suite.Update(self, data)
        for hypo in self.Values():
            hypo.Update(data)
            
    def Likelihood(self, data, hypo):
        dirichlet = hypo
        like = 0
        for i in range(1000):
            like += dirichlet.Likelihood(data)

        return like

In [25]:
ns = range(3, 30)
suite = Species(ns)

- Assumimos que qualquer valor nesse intervalo é igualmente provável.
- O comprimento de `data` é o número de espécies observadas. 
- Se vemos mais espécies do que pensávamos existir, a probabilidade é 0.

## 15.4 - Amostragem aleatória

Existem duas maneiras de gerar uma amostra aleatória a partir de uma distribuição Dirichlet:
1. Usar as distribuições beta marginais;
2. Selecionar valores de n distribuições gama e normalizar dividindo pelo total

```python
# class Dirichlet

    def Random(self):
        p = numpy.random.gamma(self.params)
        return p / p.sum()
```

Distribuição a posteriori de n;

```python
    def DistOfN(self):
        pmf = thinkbayes.Pmf()
        for hypo, prob in self.Items():
            pmf.Set(hypo.n, prob)
        return pmf
```

Percorre as hipóteses de nível superior e acumula a probabilidade de cada n.

![](post_15_4.png)

Distribuição a posteriori de n

- O valor mais provável é 4;
- Valores de 3 a 7 são razoavelmente prováveis;
- A probabilidade de haver 29 espécies é baixa o suficiente para ser insignificante;

- A priori -> uniforme;
- Com mais informações sobre as espécies, poderemos escolher uma a priori diferente;

## 15.5 - Otimização

- Poucas linhas de código para a solução;
- É lento;
    - Bom para poucas espécies;
    - Não é bom para o problema do umbigo;

As próximas seções apresentam uma série de otimizações:
- Não precisamos realmente de n objetos Dirichlet; podemos armazenar os parâmetros no nível superior da hierarquia.
- Mesmo conjunto de valores aleatórios para todas as hipóteses.
- Fazer as atualizações de uma espécie por vez;
- combina as sub-hipóteses no nível superior e usa operações numpy de array para acelerar as coisas.

## 15.6 - Colapsando a hierarquia

In [28]:
class Species2(object):
    
    def __init__(self, ns):
        self.ns = ns
        self.probs = numpy.ones(len(ns), dtype=numpy.double)
        self.params = numpy.ones(self.high, dtype=numpy.int)
    
    def Update(self, data):
        like = numpy.zeros(len(self.ns), dtype=numpy.double)
        for i in range(1000):
            like += self.SampleLikelihood(data)

        self.probs *= like
        self.probs /= self.probs.sum()

        m = len(data)
        self.params[:m] += data
    
    def SampleLikelihood(self, data):
        gammas = numpy.random.gamma(self.params)

        m = len(data)
        row = gammas[:m]
        col = numpy.cumsum(gammas)

        log_likes = []
        for n in self.ns:
            ps = row / col[n-1]
            terms = data * numpy.log(ps)
            log_like = terms.sum()
            log_likes.append(log_like)

        log_likes -= numpy.max(log_likes)
        likes = numpy.exp(log_likes)

        coefs = [thinkbayes.BinomialCoef(n, m) for n in self.ns]
        likes *= coefs

        return likes

```python
__init__()
```
- `ns` é a lista de valores hipotéticos para n; 
- `probs` é a lista de probabilidades correspondentes;
- `params` é a sequência dos parâmetros de Dirichlet;

--------------------------

```python
Update()
```
Atualiza os dois níveis da hierarquia: 
1. A probabilidade de cada valor de n;
2. Os parâmetros Dirichlet:

-------------------------------

```python
SampleLikelihood()
```
- Duas oportunidades de otimização:
    - O número hipotético(n) de espécies excede o número observado(m), precisamos apenas dos primeiros m.
    - Número de espécies for grande, a probabilidade dos dados pode ser muito pequena, usamos log.

- A versão otimizada é menos legível e mais suscetível a erros do que a original.
- Usar a versão simples para fazer um teste de regressão;

## 15.7 - Mais um problema

À medida que o número de espécies observadas aumenta, leva mais iterações para convergir para uma boa resposta.

As prevalências que escolhemos da distribuição de Dirichlet;

A maioria das iterações não fornece nenhuma contribuição útil para a probabilidade total;

Solução:
- A atualizar da distribuição a priori com todo o conjunto de dados ou dividi-la em uma série de atualizações;

Essa versão representa as hipóteses como uma lista de objetos Dirichlet;


```python
# class Species4

    def Likelihood(self, data, hypo):
        dirichlet = hypo
        like = 0
        for i in range(self.iterations):
            like += dirichlet.Likelihood(data)

        # correct for the number of unseen species the new one
        # could have been
        m = len(data)
        num_unseen = dirichlet.n - m + 1
        like *= num_unseen

        return like
```

fator `num_unseen`?
    

## 15.8 - Ainda não terminamos

- A execução das atualizações de uma espécie por vez resolve um problema, mas cria outro.
    - Cada atualização leva um tempo proporcional a km.
        - k é o número de hipóteses e 
        - m é o número de espécies observadas
    - Tempo total de execução é proporcional ao km².
    

Solução
- livraremos dos objetos Dirichlet e recolheremos os dois níveis da hierarquia em um único objeto.

- O tempo de execução desta função é proporcional ao número de hipóteses, k. 
- Como são executados m vezes, o tempo de execução da atualização é proporcional a km.

## 15.9 - Os dados do umbigo

Já chega de leões, tigres e ursos. Vamos voltar aos umbigos. Para ter uma idéia da aparência dos dados, considere o sujeito B1242, cuja amostra de 400 leituras produziu 61 espécies com as seguintes contagens:

92, 53, 47, 38, 15, 14, 12, 10, 8, 7, 7, 5, 5, 
4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2,
1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
Existem algumas espécies dominantes que compõem uma grande fração do todo, mas muitas espécies que produziram apenas uma única leitura. O número desses "singletons" sugere que é provável que haja pelo menos algumas espécies invisíveis.

No exemplo de leões e tigres, assumimos que cada animal na reserva é igualmente provável de ser observado. Da mesma forma, para os dados do umbigo, assumimos que cada bactéria tem a mesma probabilidade de produzir uma leitura.

Na realidade, cada etapa do processo de coleta de dados pode apresentar vieses. É provável que algumas espécies sejam apanhadas por um cotonete ou produzam amplicons identificáveis. Então, quando falamos sobre a prevalência de cada espécie, devemos lembrar dessa fonte de erro.

Devo também reconhecer que estou usando o termo "espécie" livremente. Primeiro, espécies bacterianas não estão bem definidas. Segundo, algumas leituras identificam uma espécie específica, outras apenas identificam um gênero. Para ser mais preciso, devo dizer "unidade taxonômica operacional", ou OTU.

Agora vamos processar alguns dos dados do umbigo. Defino uma classe chamada Assunto para representar informações sobre cada sujeito no estudo:

classe Assunto (objeto):

    def __init __ (próprio, código):
        self.code = code
        self.species = []
Cada sujeito tem um código de sequência, como "B1242", e uma lista de pares (contagem, nome da espécie), classificados em ordem crescente por contagem. O Subject fornece vários métodos para facilitar o acesso a essas contagens e nomes de espécies. Você pode ver os detalhes em http://thinkbayes.com/species.py . Para mais informações, consulte a Seção  0.3 .


Figura 15.3: Distribuição de n para o sujeito B1242.
O Subject fornece um método chamado Process que cria e atualiza um conjunto Species5 , que representa as distribuições de ne as prevalências.

E o Suite2 fornece DistOfN , que retorna a distribuição posterior de n .

class Suite2

    def DistN (auto):
        items = zip (self.ns, self.probs)
        pmf = thinkbayes.MakePmfFromItems (itens)
        retornar pmf
A Figura  15.3 mostra a distribuição de n para o sujeito B1242. A probabilidade de existir exatamente 61 espécies, e nenhuma espécie invisível, é quase nula. O valor mais provável é 72, com intervalo de 90% credível de 66 a 79. Na extremidade alta, é improvável que existam 87 espécies.

Em seguida, calculamos a distribuição posterior da prevalência para cada espécie. Species2 fornece DistOfPrevalence :

class Species2

    def DistOfPrevalence (próprio, índice):
        metapmf = thinkbayes.Pmf ()

        para n, prob em zip (self.ns, self.probs):
            beta = self.MarginalBeta (n, índice)
            pmf = beta.MakePmf ()
            metapmf.Set (pmf, prob)

        mix = thinkbayes.MakeMixture (metapmf)
        retornar metapmf, misturar
O índice indica quais espécies queremos. Para cada n , temos uma distribuição posterior diferente de prevalência.


Figura 15.4: Distribuição das prevalências para o sujeito B1242.
O loop percorre os possíveis valores de n e suas probabilidades. Para cada valor de n , obtém um objeto Beta que representa a distribuição marginal para as espécies indicadas. Lembre-se de que os objetos Beta contêm os parâmetros alfa e beta ; eles não têm valores e probabilidades como um Pmf, mas fornecem MakePmf , que gera uma aproximação discreta à distribuição beta contínua.

metapmf é um meta-Pmf que contém as distribuições de prevalência, condicionadas a n . MakeMixture combina o meta-Pmf em mix , que combina as distribuições condicionais em uma única distribuição de prevalência.

A Figura  15.4 mostra os resultados para as cinco espécies com mais leituras. As espécies mais prevalentes são responsáveis ​​por 23% das 400 leituras, mas como há quase certamente espécies não vistas, a estimativa mais provável para sua prevalência é de 20%, com intervalo confiável de 90% entre 17% e 23%.

## 15.10 - Distribuições preditivas

Eu introduzi o problema das espécies ocultas na forma de quatro questões relacionadas. Respondemos às duas primeiras calculando a distribuição posterior de n e a prevalência de cada espécie.

As outras duas perguntas são:

Se estamos planejando coletar leituras adicionais, podemos prever quantas novas espécies provavelmente descobriremos?
Quantas leituras adicionais são necessárias para aumentar a fração de espécies observadas para um determinado limite?
Para responder perguntas preditivas como essa, podemos usar as distribuições posteriores para simular possíveis eventos futuros e calcular distribuições preditivas para o número de espécies e fração do total, é provável que vejamos.

O núcleo dessas simulações se parece com o seguinte:

Escolha n de sua distribuição posterior.
Escolha uma prevalência para cada espécie, incluindo possíveis espécies não vistas, usando a distribuição Dirichlet.
Gere uma sequência aleatória de observações futuras.
Calcule o número de novas espécies, num_newem função do número de leituras adicionais, k .
Repita as etapas anteriores e acumule a distribuição conjunta de num_newe k .
E aqui está o código. RunSimulation executa uma única simulação:

class Assunto

    def RunSimulation (self, num_reads):
        m, visto = self.GetSeenSpecies ()
        n, observações = self.GenerateObservations (num_reads)

        curva = []
        para k, obs em enumerar (observações):
            seen.add (obs)

            num_new = len (visto) - m
            curve.append ((k + 1, num_new))

        curva de retorno
        
num_readsé o número de leituras adicionais a serem simuladas. m é o número de espécies vistas e visto é um conjunto de cadeias com um nome único para cada espécie. n é um valor aleatório da distribuição posterior e observações é uma sequência aleatória de nomes de espécies.

Cada vez que passamos pelo loop, adicionamos a nova observação à vista e registramos o número de leituras e o número de novas espécies até agora.

O resultado do RunSimulation é uma curva de rarefação , representada como uma lista de pares com o número de leituras e o número de novas espécies.

Antes de vermos os resultados, vejamos GetSeenSpecies e GenerateObservations .

class Subject

    def GetSeenSpecies (próprio):
        names = self.GetNames ()
        m = len (nomes)
        visto = conjunto (SpeciesGenerator (nomes, m))
        retorno m, visto

GetNames retorna a lista de nomes de espécies que aparecem nos arquivos de dados, mas para muitos assuntos, esses nomes não são exclusivos. Então, eu uso o SpeciesGenerator para estender cada nome com um número de série:

    def SpeciesGenerator (nomes, num):
        i = 0
        para nome nos nomes:
            rendimento '% s-% d'% (nome, i)
            i + = 1

    enquanto eu <num:
        rendimento 'invisível% d'% i
        i + = 1

Dado um nome como Corynebacterium , SpeciesGenerator produz Corynebacterium-1 . Quando a lista de nomes se esgota, produz nomes como o invisível-62 .

Aqui está GenerateObservations :

class Assunto

    def GenerateObservations (self, num_reads):
        n, prevalências = self.suite.SamplePosterior ()

        names = self.GetNames ()
        name_iter = SpeciesGenerator (nomes, n)

        d = dict (zip (name_iter, prevalências))
        cdf = thinkbayes.MakeCdfFromDict (d)
        observações = cdf.Sample (num_reads)

        return n, observações

Novamente, num_readsé o número de leituras adicionais a serem geradas. n e prevalência são amostras da distribuição posterior.

cdf é um objeto Cdf que mapeia nomes de espécies, incluindo o invisível, com probabilidades acumuladas. Usar um Cdf torna eficiente gerar uma sequência aleatória de nomes de espécies.

Finalmente, aqui está Species2.SamplePosterior :

    def SamplePosterior (auto):
        pmf = self.DistOfN ()
        n = pmf.Random ()
        prevalências = self.SamplePrevalences (n)
        retorno n, prevalências

E SamplePrevalences , que gera uma amostra de prevalências condicionadas em n :

class Species2

    def SamplePrevalences (self, n):
        params = self.params [: n]
        gammas = numpy.random.gamma (parâmetros)
        gammas / = gammas.sum ()
        retornar gammas

Vimos esse algoritmo para gerar valores aleatórios a partir de uma distribuição Dirichlet na Seção  15.4 .

A Figura  15.5 mostra 100 curvas de rarefação simuladas para o sujeito B1242. As curvas são "tremidas"; ou seja, eu mudei cada curva por um deslocamento aleatório para que elas não se sobreponham. Por inspeção, podemos estimar que, após mais 400 leituras, provavelmente encontraremos 2 a 6 novas espécies.

## 15.11 - Articulação posterior

Podemos usar essas simulações para estimar a distribuição conjunta de eknum_new e , a partir disso, podemos obter a distribuição de condicionada em qualquer valor de k . num_new

    def MakeJointPredictive (curvas):
        joint = thinkbayes.Joint ()
        para curva em curvas:
            para k, num_new na curva:
                joint.Incr ((k, num_new))
        joint.Normalize ()
        junta de retorno
MakeJointPredictive cria um objeto Joint, que é um Pmf cujos valores são tuplas.

curvas é uma lista de curvas de rarefação criadas pelo RunSimulation . Cada curva contém uma lista de pares de k e num_new.

A distribuição conjunta resultante é um mapa de cada par para sua probabilidade de ocorrer. Dada a distribuição conjunta, podemos usar Joint.Conditional para obter a distribuição de num_newcondicionado em k (consulte a Seção  9.6 ).

Subject.MakeConditionals pega uma lista de ks e calcula a distribuição condicional de num_new cada k . O resultado é uma lista de objetos Cdf.

    def MakeConditionals (curvas, ks):
        joint = MakeJointPredictive (curvas)

        cdfs = []
        para k em ks:
            pmf = joint.Condicional (1, 0, k)
            pmf.name = 'k =% d'% k
            cdf = pmf.MakeCdf ()
            cdfs.append (cdf)

        retornar cdfs
A Figura  15.6 mostra os resultados. Após 100 leituras, o número médio previsto de novas espécies é 2; o intervalo de 90% credível é de 0 a 5. Após 800 leituras, esperamos ver de 3 a 12 novas espécies.

## 15.12 - Cobertura

A última pergunta que queremos responder é: "Quantas leituras adicionais são necessárias para aumentar a fração de espécies observadas para um determinado limite?"

Para responder a essa pergunta, precisamos de uma versão do RunSimulation que calcule a fração das espécies observadas em vez do número de novas espécies.

class Assunto

    def RunSimulation (self, num_reads):
        m, visto = self.GetSeenSpecies ()
        n, observações = self.GenerateObservations (num_reads)

        curva = []
        para k, obs em enumerar (observações):
            seen.add (obs)

            frac_seen = len (visto) / float (n)
            curve.append ((k + 1, frac_seen))

        curva de retorno
Em seguida, percorremos cada curva e criamos um dicionário, d , que mapeia do número de leituras adicionais, k , para uma lista de fracs ; isto é, uma lista de valores para a cobertura alcançada após k leituras.

    def MakeFracCdfs (auto, curvas):
        d = {}
        para curva em curvas:
            para k, frac na curva:
                d.setdefault (k, []). append (frac)

        cdfs = {}
        para k, fracs em d.iteritems ():
            cdf = thinkbayes.MakeCdfFromList (fracs)
            cdfs [k] = cdf

        retornar cdfs
Então, para cada valor de k , fazemos um Cdf de fracs ; este Cdf representa a distribuição da cobertura após k leituras.

Lembre-se de que o CDF indica a probabilidade de cair abaixo de um determinado limite, portanto, o CDF complementar indica a probabilidade de excedê-lo. A Figura  15.7 mostra CDFs complementares para uma faixa de valores de k .

Para ler esta figura, selecione o nível de cobertura que você deseja obter ao longo do eixo x . Como exemplo, escolha 90%.

Agora você pode ler o gráfico para encontrar a probabilidade de atingir 90% de cobertura após k leituras. Por exemplo, com 200 leituras, você tem cerca de 40% de chance de obter 90% de cobertura. Com 1000 leituras, você tem 90% de chance de obter 90% de cobertura.

Com isso, respondemos às quatro perguntas que compõem o problema de espécies invisíveis. Para validar os algoritmos deste capítulo com dados reais, tive que lidar com mais alguns detalhes. Mas este capítulo já é muito longo, então não vou discuti-los aqui.

Você pode ler sobre os problemas e como eu os resolvi em http://allendowney.blogspot.com/2013/05/belly-button-biodiversity-end-game.html .

Você pode fazer o download do código neste capítulo em http://thinkbayes.com/species.py . Para mais informações, consulte a Seção  0.3 .